In [63]:
import pandas as pd
import numpy as np
from scipy.stats import entropy

In [64]:
X = [4, 4]
entropy(X, base=2)

1.0

In [65]:
EXAMPLE_1 = '../data/lec9/Example1.xlsx'
tr_df = pd.read_excel(EXAMPLE_1, sheet_name='Train')

In [66]:
tr_df.head()

,Instance,Outlook,Temperature,Humidity,Windy,Play
0,1,sunny,hot,high,False,no
1,2,sunny,hot,high,True,no
2,3,overcast,hot,high,False,yes
3,4,rainy,mild,high,False,yes
4,5,rainy,cool,normal,False,yes


In [121]:
all_ent = entropy(tr_df['Play'].value_counts(), base=2)
all_ent

0.940285958670631

In [122]:
sunny = tr_df[tr_df['Outlook'] == 'sunny']
rainy = tr_df[tr_df['Outlook'] == 'rainy']
overcast = tr_df[tr_df['Outlook'] == 'overcast']

print('Sunny:', entropy(sunny['Play'].value_counts(), base=2))
print('Rainy:', entropy(rainy['Play'].value_counts(), base=2))
print('Overcast:', entropy(overcast['Play'].value_counts(), base=2))

Sunny: 0.9709505944546688
Rainy: 0.9709505944546688
Overcast: 0.0


In [123]:
(5 * 0.97 + 5 * 0.97) / 14

0.6928571428571428

In [130]:
def get_ent(indx, all_ent):
    pivot = tr_df.pivot_table(index=indx, columns='Play', values='Instance', aggfunc='count', fill_value=0)
    e = []
    for i, row in pivot.iterrows():
        e.append(entropy(row, base=2))

    pivot['Entropy'] = e
    pivot['sum'] = pivot['yes'] + pivot['no']
    pivot['entropy*n'] = pivot['sum'] * pivot['Entropy']
    ent = pivot['entropy*n'].sum() / pivot['sum'].sum()
    print(pivot)
    print('Entropy by {}: {}'.format(indx, ent))
    print('Information gain by {}: {}'.format(indx, (all_ent - ent)))
    print('\n')
    return ent

In [131]:
feat = tr_df.columns[1: -1]
_all_vars = [get_ent(x, all_ent) for x in feat]

_min_ent = min(_all_vars)

print('Attribute with highest information gain: {}, {}'.format(feat[_all_vars.index(_min_ent)], (all_ent - _min_ent)))


Play      no  yes   Entropy  sum  entropy*n
Outlook                                    
overcast   0    4  0.000000    4   0.000000
rainy      2    3  0.970951    5   4.854753
sunny      3    2  0.970951    5   4.854753
Entropy by Outlook: 0.693536138896192
Information gain by Outlook: 0.246749819774439


Play         no  yes   Entropy  sum  entropy*n
Temperature                                   
cool          1    3  0.811278    4   3.245112
hot           2    2  1.000000    4   4.000000
mild          2    4  0.918296    6   5.509775
Entropy by Temperature: 0.9110633930116763
Information gain by Temperature: 0.029222565658954758


Play      no  yes   Entropy  sum  entropy*n
Humidity                                   
high       4    3  0.985228    7   6.896597
normal     1    6  0.591673    7   4.141709
Entropy by Humidity: 0.7884504573082894
Information gain by Humidity: 0.15183550136234159


Play   no  yes   Entropy  sum  entropy*n
Windy                                   
False   2